# Build a variational autoencoder

### References

https://blog.keras.io/building-autoencoders-in-keras.html

https://towardsdatascience.com/understanding-variational-autoencoders-vaes-f70510919f73

In [1]:
import sys
import numpy as np
import keras
from keras import regularizers
from keras import layers
from keras import backend
import random

if '..' not in sys.path:
    sys.path.append('..')
from car_reg_generator.car_reg_generator.uk_reg import UkRegGenerator
from car_reg_generator.car_reg_generator.uk_reg import UkRegBowVectorizer

### Data generation

In [2]:
n_train = 10000
n_val = 1000
n_test = 1000

random.seed(0)
g = UkRegGenerator()
v = UkRegBowVectorizer()

train_strs = [g.get_reg() for _ in range(n_train)]
train_vecs = np.array([v.vectorize(x) for x in train_strs])
val_strs = [g.get_reg() for _ in range(n_val)]
val_strs = [x for x in val_strs if x not in set(train_strs)]  # sanity check
val_vecs = np.array([v.vectorize(x) for x in val_strs])
test_strs = [g.get_reg() for _ in range(n_test)]
test_strs = [x for x in test_strs if x not in set(train_strs) | set(val_strs)]  # sanity check
test_vecs = np.array([v.vectorize(x) for x in test_strs])

vec_length = len(train_vecs[0])
print('input vector length = ' + str(vec_length))
print('actual number of validation samples = ' + str(len(val_strs)))
print('actual number of test samples = ' + str(len(test_strs)))

input vector length = 252
actual number of validation samples = 1000
actual number of test samples = 1000


### Variational autoencoder

In [3]:
latent_dim = 100
intermediate_dim = 500
original_dim = vec_length

### Define the encoder
inputs = keras.Input(shape=(original_dim,))
h = layers.Dense(intermediate_dim, activation='relu')(inputs)
h = layers.Dense(intermediate_dim, activation='relu')(h)
h = layers.Dense(intermediate_dim, activation='relu')(h)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_sigma = args
    epsilon = backend.random_normal(shape=(backend.shape(z_mean)[0], latent_dim),
                                    mean=0., stddev=0.1)
    return z_mean + backend.exp(z_log_sigma) * epsilon

z = layers.Lambda(sampling)([z_mean, z_log_sigma])

encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

### Define the decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
# x = layers.Dense(intermediate_dim, activation='relu')(x)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

### Define the loss function
reconstruction_loss = keras.losses.binary_crossentropy(inputs, outputs)
reconstruction_loss *= original_dim
kl_loss = 1 + z_log_sigma - backend.square(z_mean) - backend.exp(z_log_sigma)
kl_loss = backend.sum(kl_loss, axis=-1)
kl_loss *= -0.5
vae_loss = backend.mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

Do some training

In [4]:
vae.fit(train_vecs, train_vecs,
        epochs=200,
        batch_size=256,
        shuffle=True,
        validation_data=(val_vecs, val_vecs))

Epoch 1/200
40/40 [==============================] - 3s 78ms/step - loss: 132.3864 - val_loss: 100.8485
Epoch 2/200
40/40 [==============================] - 3s 71ms/step - loss: 87.9955 - val_loss: 78.4801
Epoch 3/200
40/40 [==============================] - 3s 72ms/step - loss: 73.1512 - val_loss: 68.6059
Epoch 4/200
40/40 [==============================] - 3s 70ms/step - loss: 65.4860 - val_loss: 62.7420
Epoch 5/200
40/40 [==============================] - 3s 75ms/step - loss: 60.6025 - val_loss: 58.8638
Epoch 6/200
40/40 [==============================] - 3s 75ms/step - loss: 57.0643 - val_loss: 55.6376
Epoch 7/200
40/40 [==============================] - 3s 73ms/step - loss: 54.1531 - val_loss: 52.8640
Epoch 8/200
40/40 [==============================] - 3s 71ms/step - loss: 51.6801 - val_loss: 50.6258
Epoch 9/200
40/40 [==============================] - 3s 74ms/step - loss: 49.6682 - val_loss: 48.8165
Epoch 10/200
40/40 [==============================] - 3s 74ms/step - loss: 47.85

Epoch 76/200
40/40 [==============================] - 3s 74ms/step - loss: 11.5743 - val_loss: 11.9320
Epoch 77/200
40/40 [==============================] - 3s 68ms/step - loss: 11.4632 - val_loss: 11.7867
Epoch 78/200
40/40 [==============================] - 3s 77ms/step - loss: 11.3415 - val_loss: 11.6307
Epoch 79/200
40/40 [==============================] - 3s 67ms/step - loss: 11.2280 - val_loss: 11.5091
Epoch 80/200
40/40 [==============================] - 3s 67ms/step - loss: 11.1010 - val_loss: 11.4146
Epoch 81/200
40/40 [==============================] - 3s 75ms/step - loss: 10.9779 - val_loss: 11.3841
Epoch 82/200
40/40 [==============================] - 3s 70ms/step - loss: 10.8688 - val_loss: 11.2341
Epoch 83/200
40/40 [==============================] - 3s 67ms/step - loss: 10.7708 - val_loss: 11.0899
Epoch 84/200
40/40 [==============================] - 3s 74ms/step - loss: 10.6592 - val_loss: 10.9158
Epoch 85/200
40/40 [==============================] - 3s 73ms/step - loss

40/40 [==============================] - 4s 93ms/step - loss: 6.1924 - val_loss: 6.4991
Epoch 157/200
40/40 [==============================] - 3s 70ms/step - loss: 6.1493 - val_loss: 6.4356
Epoch 158/200
40/40 [==============================] - 3s 71ms/step - loss: 6.1102 - val_loss: 6.3303
Epoch 159/200
40/40 [==============================] - 3s 72ms/step - loss: 6.0783 - val_loss: 6.2652
Epoch 160/200
40/40 [==============================] - 3s 70ms/step - loss: 6.0251 - val_loss: 6.3672
Epoch 161/200
40/40 [==============================] - 3s 70ms/step - loss: 5.9906 - val_loss: 6.2482
Epoch 162/200
40/40 [==============================] - 3s 67ms/step - loss: 5.9619 - val_loss: 6.2167
Epoch 163/200
40/40 [==============================] - 3s 69ms/step - loss: 5.9192 - val_loss: 6.1999
Epoch 164/200
40/40 [==============================] - 3s 69ms/step - loss: 5.9070 - val_loss: 6.1599
Epoch 165/200
40/40 [==============================] - 3s 73ms/step - loss: 5.8788 - val_loss: 6

Evaluation

In [5]:
decoded_regs = vae.predict(test_vecs)
print(test_strs[:10])
print([v.recover(x) for x in decoded_regs[:10]])
acc = np.sum([v.recover(x) == y for x, y in zip(decoded_regs, test_strs)]) / len(test_strs)
print('accuracy = ' + str(acc))

['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
['MY33WSZ', 'WE83GXF', 'LU19JLV', 'KX41GVE', 'WP91NCV', 'SG47LFF', 'HB17WIY', 'CA89AXN', 'GT57AKA', 'CD77SJT']
accuracy = 0.993


Use decoder as generator

In [9]:
generated_regs = decoder.predict(np.random.random((5, latent_dim)))
print([v.recover(x) for x in generated_regs])

['CY89GBA', 'YC86G0V', 'YC89VMV', 'YC86GKV', 'FC86ZWA']
